In [61]:
from pymongo.mongo_client import MongoClient
import yfinance as yf
import os
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
client = MongoClient(os.environ["ATLAS_URI"])

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [8]:
print(client.list_database_names())

['sample_mflix', 'admin', 'local']


In [9]:
db = client["fomo_factory"]

Collection(Database(MongoClient(host=['ac-9peyuts-shard-00-01.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-02.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-00.tfgxdt6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='stockCluster', authsource='admin', replicaset='atlas-emribc-shard-0', ssl=True), 'fomo_factory'), 'stock_prices')

In [37]:
db.drop_collection("stock_prices")
db.create_collection(
    "stock_prices",
    timeseries={
        "timeField": "timestamp",
        "metaField": "metadata",
        "granularity": "minutes"
    },
    expireAfterSeconds=60*60*24*7
)

Collection(Database(MongoClient(host=['ac-9peyuts-shard-00-01.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-02.tfgxdt6.mongodb.net:27017', 'ac-9peyuts-shard-00-00.tfgxdt6.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='stockCluster', authsource='admin', replicaset='atlas-emribc-shard-0', ssl=True), 'fomo_factory'), 'stock_prices')

In [33]:
stock_data_aapl=yf.Ticker("AAPL").history(period="5d", interval="1m")
stock_data_msft=yf.Ticker("MSFT").history(period="5d", interval="1m")


In [38]:
time_series_array=[]
for index, row in stock_data_msft.iterrows():
    time_series_array.append({
        "timestamp": index,
        "metadata": {
            "symbol": "AAPL",
        },
        "price":row["Close"],
    })

for index, row in stock_data_aapl.iterrows():
    time_series_array.append({
        "timestamp": index,
        "metadata": {
            "symbol": "MSFT",
        },
        "price":row["Close"],
    })

In [39]:
db["stock_prices"].insert_many(time_series_array)

In [36]:
stock_data_aapl

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2024-07-08 09:30:00-04:00,227.089996,227.350006,226.470001,226.922501,2348656,0.0,0.0
2024-07-08 09:31:00-04:00,226.949997,227.149994,226.550003,226.580002,395110,0.0,0.0
2024-07-08 09:32:00-04:00,226.559998,226.970001,226.539993,226.880005,282368,0.0,0.0
2024-07-08 09:33:00-04:00,226.853394,226.979996,226.490005,226.639999,344988,0.0,0.0
2024-07-08 09:34:00-04:00,226.645004,226.880005,226.619995,226.839996,317436,0.0,0.0
...,...,...,...,...,...,...,...
2024-07-12 15:55:00-04:00,231.020004,231.026993,230.389999,230.529999,254676,0.0,0.0
2024-07-12 15:56:00-04:00,230.529999,230.679993,230.500000,230.529999,159098,0.0,0.0
2024-07-12 15:57:00-04:00,230.539902,230.544998,230.360001,230.375000,166645,0.0,0.0


In [51]:
cursor=db["stock_prices"].find(
    {
        "metadata.symbol": "AAPL",
        "timestamp": {
            "$gte": stock_data_aapl.index[0],
            "$lte": stock_data_aapl.index[10]
        }
    },
    {
        "timestamp": 1,
        "price": 1,
        "_id": 0
    }
)

In [52]:
results = list(cursor)
for document in results:
    print(document)

{'timestamp': datetime.datetime(2024, 7, 8, 13, 30), 'price': 465.6400146484375}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 31), 'price': 465.80999755859375}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 32), 'price': 466.364013671875}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 33), 'price': 466.20001220703125}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 34), 'price': 465.2850036621094}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 35), 'price': 465.625}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 36), 'price': 465.20001220703125}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 37), 'price': 466.2850036621094}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 38), 'price': 466.7799987792969}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 39), 'price': 466.6199951171875}
{'timestamp': datetime.datetime(2024, 7, 8, 13, 40), 'price': 466.239990234375}
